# 加载数据

In [ ]:
import os
import torch
from torch.utils.data import DataLoader
from torch import nn, optim
from torchvision.transforms import transforms
from torchsummary import summary

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Mon Oct 18 06:22:47 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !unzip /content/drive/MyDrive/Data.zip


In [ ]:
!ls /content/drive/MyDrive/Data/data/train -l | grep "^-" | wc -l
!ls /content/drive/MyDrive/Data/data/test -l | grep "^-" | wc -l

1534
384


In [ ]:
%cd /content
if os.path.exists("Ultrasound"):
    %rm -rf Ultrasound
!ls
if not os.path.exists("Ultrasound"):
    !git clone https://github.com/z872845991/Ultrasound.git
%cd Ultrasound
!ls

In [ ]:
device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


# 参数

In [ ]:
from dataset.Fetus import FetusDataset
from model.unet_res_myself_pool_connect_o_C import Unet_res_myself_pool
from model.unet import Unet
from model.idea3 import Idea3
from tools.metrics import dice_coef,iou_score,get_accuracy,get_precision,get_specificity,get_recall,get_F1
from tools.utils import AverageMeter
from train.train_unet_local import train_model_local
import matplotlib.pyplot as plt
import matplotlib

In [ ]:
num_epoch=81
batch_size={'train':2,
            'val':1
            }
train_path='/content/Data/data/train'
val_path='/content/Data/data/test'
#变换
x_transforms = transforms.Compose([
    transforms.Resize((512,512)),
    # transforms.CenterCrop(512),
    transforms.ToTensor()
    # transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])
# mask只需要转换为tensor
y_transforms = transforms.Compose([
    transforms.Resize((512, 512)),
    # transforms.CenterCrop(512),
    transforms.ToTensor()
   ])
#加载数据集
train_dataset=FetusDataset(train_path,transform=x_transforms,target_transform=y_transforms)
val_dataset=FetusDataset(val_path,transform=x_transforms,target_transform=y_transforms)
#设置dataloader
dataloaders={
    'train':DataLoader(train_dataset,batch_size=batch_size['train'],shuffle=True,num_workers=0),
    'val':DataLoader(val_dataset,batch_size=batch_size['val'],shuffle=True,num_workers=0)
             }



# 训练

## 模型数据1

In [ ]:
model=Idea3(n_class=1)
#model=Unet(n_class=1)
model.to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters())

train=train_model_local(model)
train.summarys(input_size=(3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
       BatchNorm2d-2         [-1, 64, 224, 224]             128
              ReLU-3         [-1, 64, 224, 224]               0
            Conv2d-4         [-1, 64, 224, 224]          36,928
       BatchNorm2d-5         [-1, 64, 224, 224]             128
              ReLU-6         [-1, 64, 224, 224]               0
         MaxPool2d-7         [-1, 64, 112, 112]               0
            Conv2d-8        [-1, 128, 112, 112]          73,856
       BatchNorm2d-9        [-1, 128, 112, 112]             256
             ReLU-10        [-1, 128, 112, 112]               0
           Conv2d-11        [-1, 128, 112, 112]         147,584
      BatchNorm2d-12        [-1, 128, 112, 112]             256
             ReLU-13        [-1, 128, 112, 112]               0
        MaxPool2d-14          [-1, 128,

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [ ]:
train.compile(dataloaders,criterion,optimizer,num_epoch,batch_size,train_path,val_path,device)

In [ ]:
file1='/content/drive/MyDrive/result/train/train_7m_Idea3_change.txt'
file2='/content/drive/MyDrive/result/test/test_7m_Idea3_change.txt'
path1='/content/drive/MyDrive/result/checkpoins/train_7m_Idea3'
train.fit(file1,file2,path1)

In [ ]:
pic1=next(iter(dataloaders['val']))
pth1=model.load_state_dict(torch.load(os.path.join(path1,'_%d.pth'%(num_epoch-1)))).to(device)
y=pth1(pic1).cpu()
img1_y=torch.squeeze(y).numpy()
plt.imshow(img1_y)

## 模型数据2


In [ ]:
model2=Unet_res_myself_pool(n_class=1)
#model=Unet(n_class=1)
model2.to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model2.parameters())

train2=train_model_local(model2)
train2.summarys(input_size=(3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]             256
       BatchNorm2d-2         [-1, 64, 224, 224]             128
              ReLU-3         [-1, 64, 224, 224]               0
            Conv2d-4         [-1, 64, 224, 224]          36,928
       BatchNorm2d-5         [-1, 64, 224, 224]             128
              ReLU-6         [-1, 64, 224, 224]               0
            Conv2d-7         [-1, 64, 224, 224]          36,928
       BatchNorm2d-8         [-1, 64, 224, 224]             128
              ReLU-9         [-1, 64, 224, 224]               0
        MaxPool2d-10         [-1, 64, 112, 112]               0
        MaxPool2d-11         [-1, 64, 112, 112]               0
           Conv2d-12        [-1, 128, 112, 112]           8,320
      BatchNorm2d-13        [-1, 128, 112, 112]             256
             ReLU-14        [-1, 128, 1

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [ ]:
train2.compile(dataloaders,criterion,optimizer,num_epoch,batch_size,train_path,val_path,device)

In [ ]:
file3='/content/drive/MyDrive/result/train/train_7m_Unet_res_myself_pool_change.txt'
file4='/content/drive/MyDrive/result/test/test_7m_Unet_res_myself_pool_change.txt'
path2='/content/drive/MyDrive/result/checkpoints/train_7m_Unet_res_myself_pool'
train2.fit(file3,file4,path2)

train 	 epoch: 0  ,idx: 383   ,loss: 0.138   ,miou:   0.039,maxiou: 0.560    ,miniou: 0.000    ,mdice: 0.114   ,maxdice: 0.353   ,mindice: 0.018   

val 	 epoch: 0  ,idx: 191   ,miou:   0.002,maxiou: 0.345    ,miniou: 0.000    ,mdice: 0.030   ,maxdice: 0.335   ,mindice: 0.002   

train 	 epoch: 1  ,idx: 383   ,loss: 0.047   ,miou:   0.311,maxiou: 0.782    ,miniou: 0.000    ,mdice: 0.358   ,maxdice: 0.729   ,mindice: 0.019   

val 	 epoch: 1  ,idx: 191   ,miou:   0.006,maxiou: 0.618    ,miniou: 0.000    ,mdice: 0.035   ,maxdice: 0.594   ,mindice: 0.002   

train 	 epoch: 2  ,idx: 383   ,loss: 0.035   ,miou:   0.449,maxiou: 0.845    ,miniou: 0.000    ,mdice: 0.484   ,maxdice: 0.783   ,mindice: 0.010   

val 	 epoch: 2  ,idx: 191   ,miou:   0.001,maxiou: 0.197    ,miniou: 0.000    ,mdice: 0.028   ,maxdice: 0.233   ,mindice: 0.001   

train 	 epoch: 3  ,idx: 383   ,loss: 0.035   ,miou:   0.505,maxiou: 0.833    ,miniou: 0.000    ,mdice: 0.529   ,maxdice: 0.807   ,mindice: 0.003   

val 	 ep

In [ ]:
pic2=next(iter(dataloaders['val']))
pth2=model.load_state_dict(torch.load(os.path.join(path2,'_%d.pth'%(num_epoch-1)))).to(device)
y2=pth2(pic2).cpu()
img2_y=torch.squeeze(y2).numpy()
plt.imshow(img2_y)

## 模型数据3


In [ ]:
model3=Unet(n_class=1)
#model=Unet(n_class=1)
model3.to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model3.parameters())

train3=train_model_local(model3)
train3.summarys(input_size=(3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
       BatchNorm2d-2         [-1, 64, 224, 224]             128
              ReLU-3         [-1, 64, 224, 224]               0
            Conv2d-4         [-1, 64, 224, 224]          36,928
       BatchNorm2d-5         [-1, 64, 224, 224]             128
              ReLU-6         [-1, 64, 224, 224]               0
         MaxPool2d-7         [-1, 64, 112, 112]               0
            Conv2d-8        [-1, 128, 112, 112]          73,856
       BatchNorm2d-9        [-1, 128, 112, 112]             256
             ReLU-10        [-1, 128, 112, 112]               0
           Conv2d-11        [-1, 128, 112, 112]         147,584
      BatchNorm2d-12        [-1, 128, 112, 112]             256
             ReLU-13        [-1, 128, 112, 112]               0
        MaxPool2d-14          [-1, 128,

In [ ]:
train3.compile(dataloaders,criterion,optimizer,num_epoch,batch_size,train_path,val_path,device)

In [ ]:
file5='/content/drive/MyDrive/result/train/train_7m_Unet_change.txt'
file6='/content/drive/MyDrive/result/test/test_7m_Unet_change.txt'
path3='/content/drive/MyDrive/result/checkpoints/train_7m_Unet_change'
train3.fit(file5,file6,path3)

In [ ]:
pic3=next(iter(dataloaders['val']))
pth3=model.load_state_dict(torch.load(os.path.join(path3,'_%d.pth'%(num_epoch-1)))).to(device)
y3=pth3(pic3).cpu()
img3_y=torch.squeeze(y3).numpy()
plt.imshow(img3_y)